# Парсинг данных с сайта https://familio.org/ с использованием библиотеки BeautifulSoup

In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import xml.etree.ElementTree as ET
import json

In [2]:
final_data = pd.DataFrame()

In [3]:
def Familio_Parser(url): 
    global final_data
    response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
    page1 = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
    links = page1.findAll('script')
    data = pd.read_json(links[3].text)
    data1 = pd.DataFrame(data=data['props']['pageProps'])
    data2 = pd.DataFrame(data=data1['records']['hydra:member'])
    data2 = data2.drop({
        'uuid',
        'is_comment',
        'owner_level',
        'start_coord_margin_left',
        'start_coord_margin_top',
        'end_coord_margin_left',
        'end_coord_margin_top',
        'parent_id',
        'settlements',
        '@id',
        'page',
        'parent'
        }, axis=1)
    final_data = pd.concat([final_data, data2])
    return final_data

In [5]:
#urls = pd.read_csv('Vologda_URL.csv')
url_part_1 = 'https://familio.org/knowledge-base/catalogs/volog1907' # URL стартовой страницы 

In [8]:
# def Familio_Parser_gubernia(url): 
response = requests.get(url_part_1) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page1 = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
links = page1.findAll('script')

In [11]:
links[3]

<script async="" src="https://code.jivo.ru/widget/C988G4v5rm"></script>

In [ ]:
data = pd.read_json(links[3].text)
data1 = pd.DataFrame(data=data['props']['pageProps'])
data2 = pd.DataFrame(data=data1['records']['hydra:view'])

In [7]:
data1.head()

NameError: name 'data1' is not defined

In [ ]:
links[3].text

Code

In [ ]:
# словарь уездов задаем вручную
uezd_dict = {
    '01699a7c-9089-4d10-9a8e-37a1428a0359':'Грязовецкий уезд',
    '01ddacd6-16a4-40db-bd0e-c40f276c0c26':'Вологодский уезд',
    '06046cd7-7737-4360-8406-be7baba36827':'Тотемский уезд',
    '1cddca56-3054-4f72-ac83-90d298872d83':'Вельский уезд',
    '1eccb997-552e-4d0d-9ce7-51c048af7a2f':'Кадниковский уезд',
    '264de634-a385-4a33-9cd7-236c83be3e1e':'Велико-Устюгский уезд'
}

In [ ]:
def Familio_Guber_Parser(url):
    # gubernia page parsing function
    # return uezd list for 1 page
    response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
    page1 = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
    links = page1.findAll('script')
    data = pd.read_json(links[3].text)
    data1 = pd.DataFrame(data=data['props']['pageProps'])
    data2 = pd.DataFrame(data=data1['childGroups']['hydra:member'])
    if data2.shape[0] > 0:
        vol_list = data2[['uuid','name_ru']]
    else:
        vol_list = pd.DataFrame()
    return vol_list

In [ ]:
vol_df = pd.DataFrame() # пустой фрейм для списка уездов
#
uezd_uuid_list = list(uezd_dict.keys())
uezd_name_list = list(uezd_dict.values())
#
for u in range(len(uezd_uuid_list)):
    # цикл по списку уездов
    for i in range(3): # цикл по страницам со списком волостей уезда
        url_part_2 = '&page-group=' + str(i+1) + '&level1='
        url = url_part_1 + url_part_2 + uezd_uuid_list[u]
        vol_df_ = Familio_Guber_Parser(url)
        vol_df_['uezd_name'] = uezd_name_list[u]
        vol_df_['uezd_uuid'] = uezd_uuid_list[u]
        vol_df_['uezd_url'] = url
        if not vol_df_.empty:
            vol_df = pd.concat([vol_df, vol_df_], axis=0, ignore_index=True)

# в итоге получаем датафрейм со списком волостей по уезду

In [ ]:
vol_df.head()

# 2 level

In [ ]:
# посчитаем сколько страниц для каждой вол
# https://familio.org/knowledge-base/catalogs/nmyg1901?page-records=1&page-group=1&level1=528ff402-3d0c-4d21-8db2-c384f6a1271b&level2=00902776-d347-4996-af91-8a45177d3d54
vol_count = vol_df.shape[0] # volost quantity
vol_df['pages'] = 0 # создаем столбец под количество страниц в волости
for v in range(vol_count):
    # cycle in volost list
    url = url_part_1+'?page-group=1&page-records=1&level1=' + vol_df['uezd_uuid'].loc[v] + '&level2=' + vol_df['uuid'].loc[v]
    response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
    page1 = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
    links = page1.findAll('script')
    data = pd.read_json(links[3].text)
    data1 = pd.DataFrame(data=data['props']['pageProps'])
    a = data1['currentGroupInfo']['settlement_count']
    if a % 10 == 0:
        b = a // 10
    else:
        b = a // 10 + 1
    vol_df['pages'].loc[v] = b # заполняем столбец значениями

In [ ]:
final_data = pd.DataFrame() # подготовим пустой датафрейм для итогового списка

In [ ]:
def Familio_Uezd_Parser(url):
    # функция парсит страницу и заполняет датафрейм
    global final_data
    response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
    page1 = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
    links = page1.findAll('script')
    data = pd.read_json(links[3].text)
    data1 = pd.DataFrame(data=data['props']['pageProps'])
    if not data1.empty:
        data2 = pd.DataFrame(data=data1['records']['hydra:member'])
        if not data2.empty:
            data2 = data2.drop({
            'start_coord_margin_left',
            'start_coord_margin_top',
            'end_coord_margin_left',
            'end_coord_margin_top',
            '@id',
            'page',
            'parent'
            }, axis=1)
            final_data = pd.concat([final_data, data2])

In [ ]:
vol_count = vol_df.shape[0] # volost quantity
for v in range(vol_count):
    # cycle in volost list
    for i in range(vol_df['pages'].loc[v]):
        # cycle in pages of volost
        url = url_part_1 + vol_df['uezd_uuid'].loc[v] + '&page-records=' + str(i+1) + '&page-group=1&level2='+ vol_df['uuid'].loc[v] + '&catalogUUID=nmyg1901'
        Familio_Uezd_Parser(url)

# в итоге получаем DF со списком np

In [ ]:
final_data.to_csv('data_vologda.csv') # выгрузка в csv файл

try

In [ ]:
# prepare urls list for each page of villages
urls_list = []
vol_count = vol_df.shape[0] # volost quantity
for v in range(vol_count):
    # cycle in volost list
    url = url_part_1 + vol_df['uezd_uuid'].loc[v] + '&page-records=1&page-group=1&level2='+ vol_df['uuid'].loc[v] + '&catalogUUID=nmyg1901'
    urls_list.append(url)